<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_6_Accuracy_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this module, we will import all of our flood maps and conduct an accuracy assessment. We will create confusion matrices which will show True Positives (TP), False Positives(FP), False Negatives(FN), and True Negatives.

* TP = True Positive. Water in reference data, water in flood map.
* FP = False Positive. Nonwater in reference data, water in flood map.
* FN = False Negative. Water in Reference Data, nonwater in flood map.
* TN = True Negative. Nonwater in reference data, nonwater in flood map.

From the confusion matrix, we can calculate the following statistics (formulas and interpretation from Landwehr et al 2024 (See References):

* Overall Accuracy = $\begin{equation} \frac{TP + TN}{TP + FP + FN + TN} \end{equation} $
  * Proportion of correctly classified instances (regardsless of class) to the total number of instances
* Recall (aka Completeness, Sensitivity, or Producer's Accuracy for Water) = $\begin{equation} \frac{TP}{TP+FN} \end{equation} $
  * Fraction of actual positive instances that are correctly predicted
* Precision (aka aka Correctness aka Positive Predict Value aka User's Accuracy for Water) = $\begin{equation} \frac{TP}{TP + FP} \end{equation} $
  * Fraction of positive predictions that are actually correct.
* Specificity = Producer's Accuracy for Nonwater = $\begin{equation} \frac{TN}{TN+FP} \end{equation} $
* Negative Predictive Value = User's Accuracy for Nonwater = $\begin{equation} \frac{TN}{TN+FN} \end{equation} $
* F1 score (aka Dice Coefficient) =  $\begin{equation} \frac{2 * TP}{2 * TP + FP + FN} \end{equation} $
  * combines precision and recall to a harmonic mean
* Normalized Matthews Correlation Coefficient (aka Phi coefficient, mean square coefficiency coefficient) = $\begin{equation} \frac{\frac{TP * TN - FP * FN}{\sqrt{(TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)}}}{2} \end{equation} $
  * Correlation between observed and predicted classes. Useful for cases where there is a large class imbalance.

We will calculate the confusion matrices and above metrics for all points, for high confidence reference points, and for various slope, elevation, and landcover classes.

We will then write the results of the accuracy assessmnent to a CSV.

# Step 1: Import packages

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import ee
import geemap
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
ee.Authenticate()

ee.Initialize(project='servir-sco-assets')

# MODIFIABLE VARIABLE ALERT

In [ ]:
my_gee_folder = "users/mickymags/flood_intercomparison_chad_09_26_take2/"
aoi = ee.FeatureCollection(my_gee_folder + "aoi")
doi = "2024-09-26"

In [ ]:
roi = aoi.geometry()
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

# Step 2: Import Flood Maps

In [ ]:
#!pip install geemap

Import each of the flood maps we exported at the end of Module 6, and the reference data we collected in Collect Earth Online.  

In [ ]:
#dswx = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/dswx/dswx_mosaic")
#vfm = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/vfm/vfm_mosaic")
#mcdwd= ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/mcdwd/mcdwd_mosaic")
#gfm = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfm/gfm_mosaic")
#hf = ee.Image("users/mickymags/fmi/hf_pk_06162023")
#hydrosar = ee.Image(parent_directory + '/hydrosar/hydrosar_mosaic')

In [ ]:
my_ref_data = ee.FeatureCollection(my_gee_folder + "reference_data")
dswx_hls = ee.Image(my_gee_folder + "dswxhls_harmonized")
dswx_s1 = ee.Image(my_gee_folder + "dswxs1_harmonized")
mcdwd = ee.Image(my_gee_folder + "mcdwd_harmonized")
hf = ee.Image(my_gee_folder + "hydrafloods_harmonized")
hydrosar = ee.Image(my_gee_folder + "hydrosar_harmonized")
gfm = ee.Image(my_gee_folder + "gfm_harmonized")
vfm = ee.Image(my_gee_folder + "vfm_harmonized")

In [ ]:
# Get the projection and scale from one of the products
my_projection = hf.projection()
my_proj_string = my_projection.getInfo()['crs']
my_scale = my_projection.nominalScale().getInfo()

In [ ]:
water_viz = {
    "min": 0,
    "max": 2,
    "palette": ['D3D3D3', '000080', 'FFFFFF']
}

In [ ]:
gfm_mask = gfm.neq(2)
mcdwd_mask = mcdwd.neq(2)
vfm_mask = vfm.neq(2)
dswxhls_mask = dswx_hls.neq(2)
dswxs1_mask = dswx_s1.neq(2)
hf_mask = hf.neq(2)
hydrosar_mask = hydrosar.neq(2)

In [ ]:
gfm_final = gfm.updateMask(gfm_mask)
vfm_final = vfm.updateMask(vfm_mask)
mcdwd_final = mcdwd.updateMask(mcdwd_mask)
dswxhls_final = dswx_hls.updateMask(dswxhls_mask)
dswxs1_final = dswx_s1.updateMask(dswxs1_mask)
hydrosar_final = hydrosar.updateMask(hydrosar_mask)
hf_final = hf.updateMask(hf_mask)

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(gfm_final, water_viz, 'GFM')
Map.addLayer(vfm_final, water_viz, 'VFM')
Map.addLayer(dswxhls_final, water_viz, 'DSWx-HLS')
Map.addLayer(dswxs1_final, water_viz, 'DSWx-S1')
Map.addLayer(mcdwd_final, water_viz, 'MCDWD')
Map.addLayer(hf_final, water_viz, 'HYDRAFloods')
Map.addLayer(hydrosar_final, water_viz, 'HydroSAR')
Map.addLayer(my_ref_data, {}, 'Reference Data')
Map.addLayerControl()
Map

Now let's bring in our reference data and flood maps.

Filter our reference data to two different feature collections, one that has points that were identified as water, and one that has points that were identified as nonwater.

In [ ]:
#first_feat = my_ref_data.first()
#first_prop = first_feat.get('water?:yes')

In [ ]:
def property_setter(feature_collection):
  fc_size = feature_collection.size().getInfo()

  feats = []
  for q in range(fc_size):
    feat_of_int = ee.Feature(feature_collection.toList(fc_size).get(q))
    property_of_int = feat_of_int.get('water?:yes')
    if q % 100 == 0:
      print(q)
    #print(property_of_int.getInfo())
    if property_of_int.getInfo() == 100:
      feat_of_int = feat_of_int.set('reference_water', 1)
      feats.append(feat_of_int)
    else:
      feat_of_int = feat_of_int.set('reference_water', 0)
      feats.append(feat_of_int)

  new_fc = ee.FeatureCollection(feats)
  return new_fc

In [ ]:
noncloud_points = my_ref_data.filter(ee.Filter.neq("water?:cloud", 100))
noncloud_points_v2 = property_setter(noncloud_points)

all_water_points = noncloud_points.filter(ee.Filter.eq("water?:yes", 100))
all_nonwater_points = noncloud_points.filter(ee.Filter.eq("water?:no", 100))

all_water_points_v2 = noncloud_points_v2.filter(ee.Filter.eq("reference_water", 1))
all_nonwater_points_v2 = noncloud_points_v2.filter(ee.Filter.eq("reference_water", 0))

In [ ]:
my_ref_data.size().getInfo()
noncloud_points.size().getInfo()

In [ ]:
noncloud_points_v2.size

In [ ]:
noncloud_points_v2.size().getInfo()
#print(all_water_points.size().getInfo())
#all_water_points_v2.size().getInfo()

In [ ]:
highconf_points = noncloud_points_v2.filter(ee.Filter.gt("confidence", 76))

In [ ]:
#mediumconf_points = noncloud_points_v2.filter(ee.Filter.gt("confidence", 51))
#mediumconf_water_points = mediumconf_points.filter(ee.Filter.eq("reference_water", 1))
#mediumconf_nonwater_points = mediumconf_points.filter(ee.Filter.eq("reference_water", 0))

In [ ]:
#lowconf_points = noncloud_points_v2.filter(ee.Filter.gt("confidence", 26))
#lowconf_water_points = lowconf_points.filter(ee.Filter.eq("reference_water", 1))
#lowconf_nonwater_points = lowconf_points.filter(ee.Filter.eq("reference_water", 0))

In [ ]:
#print(my_ref_data.size().getInfo())
#print(noncloud_points.size().getInfo())
#print(highconf_points.size().getInfo())
#print(mediumconf_points.size().getInfo())
#print(lowconf_points.size().getInfo())

In [ ]:
#Water visualization Parameters
water_vp = {'min': 0,
          'max': 2,
          'palette': ['000000', '0000FF', 'FFFFFF']
}

In [ ]:
plv = {
    'bands': ['b3', 'b2', 'b1'],
    'min': 0,
    'max': 2400,
    'gamma': 0.75
}

In [ ]:
#gfm_masked = gfm.updateMask(mymask)

In [ ]:
hydrafloods_renamed = hf.rename('hydrafloods_water')
hydrosar_renamed = hydrosar.rename('hydrosar_water')
gfm_renamed = gfm.rename('gfm_water')
dswxhls_renamed = dswx_hls.rename('dswxhls_water')
dswxs1_renamed = dswx_s1.rename('dswxs1_water')
mcdwd_renamed = mcdwd.rename('mcdwd_water')
vfm_renamed = vfm.rename('vfm_water')

In [ ]:
# ALL POINTS
hf_sampled_allpoints = hydrafloods_renamed.sampleRegions(noncloud_points_v2, geometries=True)
hs_sampled_allpoints = hydrosar_renamed.sampleRegions(noncloud_points_v2, geometries=True)
gfm_sampled_allpoints = gfm_renamed.sampleRegions(noncloud_points_v2, geometries=True)
dswxhls_sampled_allpoints = dswxhls_renamed.sampleRegions(noncloud_points_v2, geometries=True)
dswxs1_sampled_allpoints = dswxs1_renamed.sampleRegions(noncloud_points_v2, geometries=True)
mcdwd_sampled_allpoints = mcdwd_renamed.sampleRegions(noncloud_points_v2, geometries=True)
vfm_sampled_allpoints = vfm_renamed.sampleRegions(noncloud_points_v2, geometries=True)

# HIGH CONFIDENCE POINTS
hf_sampled_highconf = hydrafloods_renamed.sampleRegions(highconf_points, geometries=True)
hs_sampled_highconf = hydrosar_renamed.sampleRegions(highconf_points, geometries=True)
gfm_sampled_highconf = gfm_renamed.sampleRegions(highconf_points, geometries=True)
dswxhls_sampled_highconf = dswxhls_renamed.sampleRegions(highconf_points, geometries=True)
dswxs1_sampled_highconf = dswxs1_renamed.sampleRegions(highconf_points, geometries=True)
mcdwd_sampled_highconf = mcdwd_renamed.sampleRegions(highconf_points, geometries=True)
vfm_sampled_highconf = vfm_renamed.sampleRegions(highconf_points, geometries=True)

We want to write a function that takes a featurecollection as input, and calculates the confusion matrix, prints the overall accuracy and consumer accuracy for each class. The function will take as an optional parameter a property to filter for. This will allow us to examine statistics for different land cover and/or slope classes more easily.

In [ ]:
'''
def precision(tp, fp, tn, fn):
  prec = tp / (tp + fp)
  return prec
'''

In [ ]:
'''
def recall(tp, fp, tn, fn):
  reca = fp (tp + fn)
  return reca
'''

In [ ]:
'''
def f1_score(tp, fp, tn, fn):
  numerator = 2 * tp
  denominator = (2 * tp) + fp + fn
  f1 = numerator / denominator
  return f1
'''

In [ ]:
'''
def normalized_mcc(tp, fp, tn, fn):
  numerator = (tp * tn) - (fp * fn)
  denominator = np.sqrt((tp+fp) * (tp + fn) * (tn + fp) * (tn+fn))
  term = (numerator / denominator) + 1
  mcc = term / 2
  return mcc
'''

# Testing out SK Learn

In [ ]:

'''
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
clf = SVC(random_state=0)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=clf.classes_)
disp.plot()
plt.show()
'''

In [ ]:
#clf.classes_

In [ ]:
#y_test

In [ ]:
#predictions

In [ ]:
'''
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=0)
clf = SVC(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
ConfusionMatrixDisplay.from_predictions(
   y_test, y_pred)
plt.show()
'''

In [ ]:
#y_pred

In [ ]:
#y_test

In [ ]:
#import sklearn.metrics

In [ ]:
def confusion_matrix_reporter(fc, map_label, description, filter_prop = None, filter_value = None):

  references_list = []
  classifications_list = []

  classes = np.array([1, 0])
  my_display_labels = np.array(['water', 'nonwater'])
  classes_v2 = ['water', 'nonwater']

  if filter_prop != None:
    fc_new = fc.filter(ee.Filter.eq(filter_prop, filter_value))
    print('path1')
  else:
    fc_new = fc
    print('path2')
  fc_size = fc_new.size().getInfo()                           #fc.size().getInfo()
  my_feat_listed = fc_new.toList(fc_size).getInfo()                                           #fc.toList(fc.size().getInfo()).getInfo()
  for r in range(fc_size):                                       # fc.size().getInfo()
    my_feat = my_feat_listed[r]['properties']
    reference = my_feat['reference_water']
    classification = my_feat[map_label]
    test1 = classification == 0 or classification == 1
    test2 = reference == 0 or reference == 1
    if test1 and test2:
      classifications_list.append(classification)
      references_list.append(reference)

  references_array = np.array(references_list)
  classifications_array = np.array(classifications_list)

  accuracy = sklearn.metrics.accuracy_score(references_array, classifications_array)
  acc_string = "{0} accuracy: {1:0.3f}".format(description, accuracy)

  f1 = sklearn.metrics.f1_score(references_array, classifications_array)
  f1_string = "{0} f1 score: {1:0.3f}".format(description, f1)

  prec = sklearn.metrics.precision_score(references_array, classifications_array)
  prec_string = "{0} Precision (aka user's accuracy for water): {1:0.3f}".format(description, prec)

  reca = sklearn.metrics.recall_score(references_array, classifications_array)
  reca_string = "{0} Recall (aka producer's accuracy for water): {1:0.3f}".format(description, reca)

  my_cm = sklearn.metrics.confusion_matrix(references_array, classifications_array, labels = classes)
  tp = my_cm[0][0]
  fn = my_cm[0][1]
  fp = my_cm[1][0]
  tn = my_cm[1][1]

  sample_size = tp + fp + fn + tn

  specificity = tn / (tn + fp)
  spec_string = "{0} Specificity (aka producer's accuracy for nonwater): {1:0.3f}".format(description, specificity)

  npv = tn / (tn + fn)
  npv_string = "{0} Negative Predictive Value (aka nonwater user's accuracy): {1: 0.3f}".format(description, npv)

  mcc = sklearn.metrics.matthews_corrcoef(references_array, classifications_array)
  nmcc = (mcc + 1) / 2
  nmcc_string = "{0} Normalized Matthew's Correlation Coefficien: {1:0.3f}".format(description, nmcc)

  #print(my_cm)
  #print('true pos', tp)
  #print('false pos', fp)
  #print('false negative', fn)
  #print('true negative', tn)

  disp2 = ConfusionMatrixDisplay.from_predictions(references_array,
                              classifications_array,
                              labels = classes,
                              display_labels = my_display_labels,
                              cmap = plt.cm.Blues)
  print(acc_string)
  print(f1_string)
  print(prec_string)
  print(reca_string)
  print(spec_string)
  print(nmcc_string)
  stats_list = [description, sample_size, accuracy, prec, reca, specificity, npv, f1, nmcc]

  return stats_list

In [ ]:
'''
def confusion_matrix_reporter(fc, map_label, description):

  references_list = []
  classifications_list = []

  #print(type(fc))
  #first_fc = fc.first().getInfo()
  #print(first_fc)

  classes = np.array([1, 0])
  my_display_labels = np.array(['water', 'nonwater'])
  classes_v2 = ['water', 'nonwater']
  fc_size = fc.size().getInfo()
  my_feat_listed = fc.toList(fc.size().getInfo()).getInfo()
  for r in range(fc.size().getInfo()):
    my_feat = my_feat_listed[r]['properties']
    reference = my_feat['reference_water']
    classification = my_feat[map_label]
    test1 = classification == 0 or classification == 1
    test2 = reference == 0 or reference == 1
    if test1 and test2:
      classifications_list.append(classification)
      references_list.append(reference)

  references_array = np.array(references_list)
  classifications_array = np.array(classifications_list)

  accuracy = sklearn.metrics.accuracy_score(references_array, classifications_array)
  acc_string = "{0} accuracy: {1:0.3f}".format(description, accuracy)

  f1 = sklearn.metrics.f1_score(references_array, classifications_array)
  f1_string = "{0} f1 score: {1:0.3f}".format(description, f1)

  prec = sklearn.metrics.precision_score(references_array, classifications_array)
  prec_string = "{0} Precision (aka user's accuracy for water): {1:0.3f}".format(description, prec)

  reca = sklearn.metrics.recall_score(references_array, classifications_array)
  reca_string = "{0} Recall (aka producer's accuracy for water): {1:0.3f}".format(description, reca)

  my_cm = sklearn.metrics.confusion_matrix(references_array, classifications_array, labels = classes)
  tp = my_cm[0][0]
  fn = my_cm[0][1]
  fp = my_cm[1][0]
  tn = my_cm[1][1]

  specificity = tn / (tn + fp)
  spec_string = "{0} Specificity (aka producer's accuracy for nonwater): {1:0.3f}".format(description, specificity)

  npv = tn / (tn + fn)
  npv_string = "{0} Negative Predictive Value (aka nonwater user's accuracy): {1: 0.3f}".format(description, npv)

  mcc = sklearn.metrics.matthews_corrcoef(references_array, classifications_array)
  nmcc = (mcc + 1) / 2
  nmcc_string = "{0} Normalized Matthew's Correlation Coefficien: {1:0.3f}".format(description, nmcc)
  #prec_sc = tp / (tp + fp)
  #f1_sc = (2 * tp) / ((2 * tp) + fp + fn)
  print(my_cm)
  print('true pos', tp)
  print('false pos', fp)
  print('false negative', fn)
  print('true negative', tn)

  #disp1 = ConfusionMatrixDisplay(confusion_matrix = my_cm,
  #                               display_labels = classes)
                                 #cmap = cm.Blues())

  disp2 = ConfusionMatrixDisplay.from_predictions(references_array,
                              classifications_array,
                              labels = classes,
                              display_labels = my_display_labels,
                              cmap = plt.cm.Blues)

  #disp2.plot()
  #plt.title(f"{description} Confusion Matrix")
  #plt.plot(disp2)
  #plt.show()
  print(acc_string)
  print(f1_string)
  #print(f1_sc)
  print(prec_string)
  #print(prec_sc)
  print(reca_string)
  print(spec_string)
  print(nmcc_string)
  stats_list = [description, accuracy, prec, reca, specificity, npv, f1, nmcc]

  return stats_list
'''

In [ ]:
#hydrafloods_allconf = confusion_matrix_reporter(hf_sampled_allpoints, 'hydrafloods_water', 'HYDRAFloods All Points')

In [ ]:
#for j in range(hf_sampled_allpoints.size().getInfo()):
#  my_feat = hf_sampled_allpoints.toList(hf_sampled_allpoints.size().getInfo()).get(j)
  #classification = hf_sampled_a


In [ ]:
#hf_sampled_allpoints.getInfo()

# End testing out SK Learn

In [ ]:
'''
def accuracy_reporter(fc_input, ref_label, map_label, description, filter_bool=False, filter_prop = None, filter_value = None):

  #fc_input will be the feature collection of points. Each point should contain a reference property and a map property.
  # Both properties should have a 1 where water was noted as present and a 0 where water was absent.
  # ref_label will be the property name in fc_input that contains the reference label of water/nonwater that we collected in Collect Earth Online.
  # map_label will be the property name in fc_input that contains the reference label of water/nonwater according to one of our flood map products.

  if filter_bool == True:   # Add filter_prop code later.
    fc_input_new = fc_input.filter(ee.Filter.eq(filter_prop, filter_value))
    matrix = fc_input_new.errorMatrix(ref_label, map_label, [1, 0])
    #tru_pos = matrix.getInfo()[0][0]
    #fal_pos = matrix.getInfo()[0][1]
    #fal_neg = matrix.getInfo()[1][0]
    #tru_neg = matrix.getInfo()[1][1]
    #oat = ((tru_pos + tru_neg) / (tru_pos + fal_pos + fal_neg + tru_neg)) * 100
    #overall_acc = matrix.accuracy().getInfo() * 100
    #producers_acc =  matrix.producersAccuracy().getInfo()
    #pa_water = producers_acc[0][0]* 100
    #pa_nonwater = producers_acc[1][0] * 100
    #users_acc = matrix.consumersAccuracy().getInfo()
    #ua_water = users_acc[0][0] * 100
    #ua_nonwater = users_acc[0][1] * 100
  else:
    matrix = fc_input.errorMatrix(ref_label, map_label, [1, 0])        # Run the errorMatrix method using the reference label and map label of the feature collection
    #overall_acc = matrix.accuracy().getInfo() * 100                    # Get the overall accuracy for this matrix in terms of percent (multiply by 100 to convert from fraction to percent)
    #tru_pos = matrix.getInfo()[0][0]
    #fal_pos = matrix.getInfo()[0][1]
    #fal_neg = matrix.getInfo()[1][0]
    #tru_neg = matrix.getInfo()[1][1]
    #oat = ((tru_pos + tru_neg) / (tru_pos + fal_pos + fal_neg + tru_neg)) * 100
    #producers_acc =  matrix.producersAccuracy().getInfo()              # Get the producer's accuracy for this matrix. This will be a list of producer's accuracy for each class (water & nonwater)
    #pa_water = producers_acc[0][0]* 100                                # Get the producer's accuracy for the water class.
    #pa_nonwater = producers_acc[1][0] * 100                            # Get the producer's accuracy for the nonwater class.
    #users_acc = matrix.consumersAccuracy().getInfo()
    #ua_water = users_acc[0][0] * 100
    #ua_nonwater = users_acc[0][1] * 100

  overall_acc = matrix.accuracy().getInfo() * 100                    # Get the overall accuracy for this matrix in terms of percent (multiply by 100 to convert from fraction to percent)
  tru_pos = matrix.getInfo()[0][0]
  fal_pos = matrix.getInfo()[0][1]
  fal_neg = matrix.getInfo()[1][0]
  tru_neg = matrix.getInfo()[1][1]
  pawater_test = (tru_pos / (tru_pos + fal_neg)) * 100
  panonwater_test = (tru_neg / (fal_pos + tru_neg)) * 100
  #oat = ((tru_pos + tru_neg) / (tru_pos + fal_pos + fal_neg + tru_neg)) * 100
  producers_acc =  matrix.producersAccuracy().getInfo()              # Get the producer's accuracy for this matrix. This will be a list of producer's accuracy for each class (water & nonwater)
  pa_water = producers_acc[0][0]* 100                                # Get the producer's accuracy for the water class.
  pa_nonwater = producers_acc[1][0] * 100                            # Get the producer's accuracy for the nonwater class.
  users_acc = matrix.consumersAccuracy().getInfo()
  ua_water = users_acc[0][0] * 100
  ua_nonwater = users_acc[0][1] * 100


  # Print the statistics
  #print(oat)
  #print("True Positives:", matrix.getInfo()[0][0])
  #print("False Positives", matrix.getInfo()[0][1])
  #print("False Negatives", matrix.getInfo()[1][0])
  #print("True Negatives:", matrix.getInfo()[1][1])
  print("{} Error Matrix: {}".format(description, matrix.getInfo()))                          # Print the error matrix.
  print("{0} Overall Accuracy: {1:0.2f} %".format(description, overall_acc))
  #print("{0} Overall Accuracy sanity check: {1:0.2f} %".format(description, oat))
  print("{0} sanity check producer's accuracy for water: {1:0.2f} %".format(description, pawater_test))
  #print("{0} sanity check producer's accuracy for nonwater: {1:0.2f} %".format(description, panonwater_test))
  print("{0} Producer's Accuracy for the Water Class: {1:0.2f} %".format(description, pa_water))
  print(producers_acc)
  #print("{0} Producer's Accuracy for the Nonwater Class: {1:0.2f} %".format(description, pa_nonwater))
  #print("{0} User's Accuracy for the Water Class: {1:0.2f} %".format(description, ua_water))
  #print("{0} User's Accuracy for the Nonwater Class: {1:0.2f} %".format(description, ua_nonwater))
'''

# Step 3: All Points Accuracy Reporting

In [ ]:
#accuracy_reporter(hf_sampled_allpoints, 'reference_water', 'hydrafloods_water', 'HYDRAFloods All Points')
hydrafloods_allconf = confusion_matrix_reporter(hf_sampled_allpoints, 'hydrafloods_water', 'HYDRAFloods All Points')

In [ ]:
#accuracy_reporter(hs_sampled_allpoints, 'reference_water', 'hydrosar_water', 'HYDROSAR ALL POINTS')
hydrosar_allconf = confusion_matrix_reporter(hs_sampled_allpoints, 'hydrosar_water', 'HYDROSAR All Points')

In [ ]:
#accuracy_reporter(gfm_sampled_allpoints, 'reference_water', 'gfm_water', 'GFM ALL POINTS')
gfm_allconf = confusion_matrix_reporter(gfm_sampled_allpoints, 'gfm_water', 'GFM All Points')

In [ ]:
#accuracy_reporter(dswxhls_sampled_allpoints, 'reference_water', 'dswxhls_water', 'DSWXHLS ALL POINTS')
dswxhls_allconf = confusion_matrix_reporter(dswxhls_sampled_allpoints, 'dswxhls_water', 'DSWX-HLs All Points')

In [ ]:
#accuracy_reporter(dswxs1_sampled_allpoints, 'reference_water', 'dswxs1_water', 'DSWx-S1 ALL POINTS')
dswxs1_allconf = confusion_matrix_reporter(dswxs1_sampled_allpoints, 'dswxs1_water', 'DSWx-S1 All Points')

In [ ]:
#accuracy_reporter(mcdwd_sampled_allpoints, 'reference_water', 'mcdwd_water', 'MCDWD ALL POINTS')
mcdwd_allconf = confusion_matrix_reporter(mcdwd_sampled_allpoints, 'mcdwd_water', 'MCDWD All Points')

In [ ]:
#accuracy_reporter(vfm_sampled_allpoints, 'reference_water', 'vfm_water', 'VFM ALL POINTS')
vfm_allconf = confusion_matrix_reporter(vfm_sampled_allpoints, 'vfm_water', 'VFM All Points')

# Step 3 Part 2: High Confidence Points

In [ ]:
#accuracy_reporter(hf_sampled_highconf, 'reference_water', 'hydrafloods_water', 'HYDRAFloods High Confidence')
hydrafloods_highconf = confusion_matrix_reporter(hf_sampled_highconf, 'hydrafloods_water', 'HYDRAFloods High Confidence')

In [ ]:
#accuracy_reporter(hs_sampled_highconf, 'reference_water', 'hydrosar_water', 'HYDROSAR High Confidence')
hydrosar_highconf = confusion_matrix_reporter(hs_sampled_highconf, 'hydrosar_water', 'HYDROSAR High Confidence')

In [ ]:
#accuracy_reporter(gfm_sampled_highconf, 'reference_water', 'gfm_water', 'GFM High Confidence')
gfm_highconf = confusion_matrix_reporter(gfm_sampled_highconf, 'gfm_water', 'GFM High Confidence')

In [ ]:
#accuracy_reporter(dswxhls_sampled_highconf, 'reference_water', 'dswxhls_water', 'DSWx-HLS High Confidence')
dswxhls_highconf = confusion_matrix_reporter(dswxhls_sampled_highconf, 'dswxhls_water', 'DSWx-HLS High Confidence')

In [ ]:
#accuracy_reporter(dswxs1_sampled_highconf, 'reference_water', 'dswxs1_water', 'DSWx-S1 High Confidence')
dswxs1_highconf = confusion_matrix_reporter(dswxs1_sampled_highconf, 'dswxs1_water', 'DSWx-S1 High Confidence')

In [ ]:
#accuracy_reporter(mcdwd_sampled_highconf, 'reference_water', 'mcdwd_water', 'MCDWD HIGH CONFIDENCE')
mcdwd_highconf = confusion_matrix_reporter(mcdwd_sampled_highconf, 'mcdwd_water', 'MCDWD High Confidence')

In [ ]:
#accuracy_reporter(vfm_sampled_highconf, 'reference_water', 'vfm_water', 'VFM HIGH CONFIDENCE')
vfm_highconf = confusion_matrix_reporter(vfm_sampled_highconf, 'vfm_water', 'VFM High Confidence')

In [ ]:
'''
def accuracy_reporter(fc_input, ref_label, map_label, description, filter_bool=False, filter_prop = None, filter_value = None):

  #fc_input will be the feature collection of points. Each point should contain a reference property and a map property.
  # Both properties should have a 1 where water was noted as present and a 0 where water was absent.
  # ref_label will be the property name in fc_input that contains the reference label of water/nonwater that we collected in Collect Earth Online.
  # map_label will be the property name in fc_input that contains the reference label of water/nonwater according to one of our flood map products.

  if filter_prop == True:   # Add filter_prop code later.
    fc_input = fc_input.filter(ee.Filter.eq(filter_prop, filter_value))
  matrix = fc_input.errorMatrix(ref_label, map_label, [1, 0])        # Run the errorMatrix method using the reference label and map label of the feature collection
  overall_acc = matrix.accuracy().getInfo() * 100                    # Get the overall accuracy for this matrix in terms of percent (multiply by 100 to convert from fraction to percent)
  producers_acc =  matrix.producersAccuracy().getInfo()              # Get the producer's accuracy for this matrix. This will be a list of producer's accuracy for each class (water & nonwater)
  pa_water = producers_acc[0][0]* 100                                # Get the producer's accuracy for the water class.
  pa_nonwater = producers_acc[1][0] * 100                            # Get the producer's accuracy for the nonwater class.
  users_acc = matrix.consumersAccuracy().getInfo()
  ua_water = users_acc[0][0] * 100
  ua_nonwater = users_acc[0][1] * 100

  # Print the statistics
  print("{} Error Matrix: {}".format(description, matrix.getInfo()))                          # Print the error matrix.
  print("{0} Overall Accuracy: {1:0.2f} %".format(description, overall_acc))
  print("{0} Producer's Accuracy for the Water Class: {1:0.2f} %".format(description, pa_water))
  print("{0} Producer's Accuracy for the Nonwater Class: {1:0.2f} %".format(description, pa_nonwater))
  print("{0} User's Accuracy for the Water Class: {1:0.2f} %".format(description, ua_water))
  print("{0} User's Accuracy for the Nonwater Class: {1:0.2f} %".format(description, ua_nonwater))
'''

In [ ]:
#accuracy_reporter(hf_s, 'water_or_not', 'hydrafloods_water', 'Hydrafloods General')

In [ ]:
#accuracy_reporter(hf_sws, 'water_or_not', 'hydrafloods_water', 'HYDRAFloods Very Low Slope', filter_bool=True,
#                  filter_prop='slope_class', filter_value=2)

# Step 4: Elevation

We want to go through the entire reference dataset, and add a column for its elevation class as well as its slope class. The elevation data used will be the Copernicus GLO30 DEM, and we will determine the slope based on the `ee.Algorithms.Terrain` method offered by Google Earth Engine.

The slopes and elevations were split into four classes each based on a statistical analysis of slopes and elevations across SERVIR regions. The code for this analysis uses open data and can be found [here](https://colab.research.google.com/drive/1TW1u7O6ha2A2puuZ0JcI5MZaLZzD7Zm0?usp=sharing)

In [ ]:
copernicus = ee.ImageCollection("COPERNICUS/DEM/GLO30")
srtm = ee.Image("USGS/SRTMGL1_003")
srtm_clip = srtm.clip(aoi)
srtm_projection = srtm_clip.projection()

In [ ]:
cop_mos = copernicus.filterBounds(aoi).mosaic().clip(aoi)
cop_mos = cop_mos.select(['DEM']).rename(['elevation']).reproject(srtm_projection)

In [ ]:
elc1 = cop_mos.select('elevation')                               #elc1 = cop_mos.select('DEM').lt(140)                            # Elevation Class 1
elc2 = cop_mos.select('elevation').gte(140).And(cop_mos.lt(300)) #elc2 = cop_mos.select('DEM').gte(140).And(cop_mos.lt(300))      # Elevation Class 2
elc3 = cop_mos.select('elevation').gte(300).And(cop_mos.lt(496)) #elc3 = cop_mos.select('DEM').gte(300).And(cop_mos.lt(496))      # Elevation Class 3
elc4 = cop_mos.select('elevation').gte(496)                      #elc4 = cop_mos.select('DEM').gte(496)                           # Elevation Class 4

In [ ]:
elc1.bandNames()

In [ ]:
vp = {
    'band': ['elevation'], #'bands': ['DEM'],
    'min': 0,
    'max': 1
}

vp2 = {
    'band': ['elevation'], #'band': ['DEM'],
    'min': 0,
    'max':4
}

In [ ]:
el1test = elc1.select('elevation').eq(1) #el1test = elc1.select('DEM').eq(1)
el2test = elc2.select('elevation').eq(1) #el2test = elc2.select('DEM').eq(1)
el3test = elc3.select('elevation').eq(1) #el3test = elc3.select('DEM').eq(1)
el4test = elc4.select('elevation').eq(1)       #el4test = elc4.select('DEM').eq(1)

el_v1 = elc1.where(el1test, ee.Image(1))
el_v2 = el_v1.where(el2test, ee.Image(2))
el_v3 = el_v2.where(el3test, ee.Image(3))
el_v4 = el_v3.where(el4test, ee.Image(4))

In [ ]:
el_v4.bandNames()

In [ ]:
el_class = el_v4.rename("elevation_class")

Now we have created a raster that has a value of 1 where there is low elevation (< 140 meters), a value of 2 where there is moderately low elevation (140 < x < 300 meters), a value of 3 where there is moderately high elevation (300 < x < 496 meters), and a value of 4 where there is very high elevation ( > 496 meters).

Now, let's use the .sampleRegions() region to add the elevation information to the map.

In [ ]:
#vfm_highconf_points = confusion_matrix_reporter(vfm_sampled_highconf, 'vfm_water', 'VFM High Confidence')

In [ ]:
hf_swe_allpoints = el_class.sampleRegions(hf_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale) # hydrafloods sampled with slope information

Now we want to find out how many elevation classes are present in our reference data

In [ ]:
feat_listed = hf_swe_allpoints.toList(hf_swe_allpoints.size().getInfo()).getInfo()

In [ ]:
el_classes_present = []

for j in range(len(feat_listed)):
  feat_mine = feat_listed[j]['properties']
  el_class = feat_mine['elevation_class']
  #print(feat_mine.keys())
  #break
  el_class = feat_mine['elevation_class']
  if el_class not in el_classes_present and el_class != None:
    el_classes_present.append(el_class)

In [ ]:
print("There is/are {0} elevation class(es) present in the sample points".format(len(el_classes_present)))
print("The elevation class(es) present are: ",el_classes_present)

For this case study, we do not have more than one elevation class present in our same points, so we will skip to the next section

# **Step 5: Slope**

Now let's do the same for the slope information

In [ ]:
cop_slope = ee.Algorithms.Terrain(cop_mos).select('slope').clip(aoi)
#cop_mod.bandNames().getInfo()

In [ ]:
slc1 = cop_slope.lt(1.39)
slc2 = cop_slope.gte(1.39).And(cop_slope.lt(3.97))
slc3 = cop_slope.gte(3.97).And(cop_slope.lt(12))
slc4 = cop_slope.gte(12)

sl1test = slc1.select('slope').eq(1)
sl2test = slc2.select('slope').eq(1)
sl3test = slc3.select('slope').eq(1)
sl4test = slc4.select('slope').eq(1)

sl_v1 = slc1.where(sl1test, ee.Image(1))
sl_v2 = sl_v1.where(sl2test, ee.Image(2))
sl_v3 = sl_v2.where(sl3test, ee.Image(3))
sl_v4 = sl_v3.where(sl4test, ee.Image(4))

In [ ]:
sl_classes = sl_v4.rename('slope_class')

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(aoi, {}, 'Area of interest')
#Map.addLayer(plum, pluvp, 'Planet Usable Data Mask')
Map.addLayer(my_ref_data, {}, 'Reference Data')
#Map.addLayer(cop_slope, {}, 'Copernicus')
#Map.addLayer(cop_mos, {'bands': ['DEM']}, 'DEM')
Map.addLayer(sl_v4, vp2, 'Elevation')

Map.addLayerControl()
Map

## Step 5 Part 1: Slope analysis of points with all confidence levels

In [ ]:
#hydrafloods sampled with elevation and slope
hf_swsl_allpoints = sl_class.sampleRegions(hf_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale) # hydrafloods sampled with slope information

In [ ]:
slopes_listed = hf_swsl_allpoints.toList(hf_swsl_allpoints.size().getInfo()).getInfo()

In [ ]:
sl_classes_present = []

for t in range(len(slopes_listed)):
  this_feat = slopes_listed[t]['properties']
  sl_class = this_feat['slope_class']
  #el_class = this_feat['slope_class']
  if sl_class not in sl_classes_present and sl_class != None:
    sl_classes_present.append(sl_class)

In [ ]:
sl_classes_present

In [ ]:
print("There is/are {0} slope class(es) present in the sample points".format(len(sl_classes_present)))
print("The slope class(es) present are: ",sl_classes_present)

Because there is more than one slope class present in the data, we can move forward with the analysis by slope.

### HYDRAFloods Slope -- All Confidences

In [ ]:
hf_lowslope_allconf = confusion_matrix_reporter(hf_swsl_allpoints, 'hydrafloods_water', 'hydrafloods slope class 1 (low slope)', filter_prop = 'slope_class', filter_value = 1)

In [ ]:
hf_modlowslope_allconf = confusion_matrix_reporter(hf_swsl_allpoints, 'hydrafloods_water', 'hydrafloods slope class 2 (moderately low slope)', filter_prop = 'slope_class', filter_value = 2)

In [ ]:
hf_modhighslope_allconf = confusion_matrix_reporter(hf_swsl_allpoints, 'hydrafloods_water', 'hydrafloods slope class 3 (moderately high slope)', filter_prop = 'slope_class', filter_value = 3)

### HYDROSAR Slope -- All Confidences

In [ ]:
# HYDROSAR Sampled with Slope Information
hs_swsl_allpoints = sl_classes.sampleRegions(hs_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale)

In [ ]:
hs_lowslo_allconf = confusion_matrix_reporter(hs_swsl_allpoints, 'hydrosar_water', 'hydrosar slope class 1 (low slope)', filter_prop = 'slope_class', filter_value = 1)

In [ ]:
hs_modlowslo_allconf = confusion_matrix_reporter(hs_swsl_allpoints, 'hydrosar_water', 'hydrosar slope class 2 (moderately low slope)',
                                           filter_prop = 'slope_class', filter_value = 2)

In [ ]:
hs_modhighslo_allconf = confusion_matrix_reporter(hs_swsl_allpoints, 'hydrosar_water', 'hydrosar slope class 3 (moderately high slope)', filter_prop = 'slope_class', filter_value = 3)

### GFM Slope -- All Confidences

In [ ]:
# GFM Sampled with Slope Information
gfm_swsl_allpoints = sl_classes.sampleRegions(gfm_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale)

In [ ]:
gfm_loslo_allconf = confusion_matrix_reporter(gfm_swsl_allpoints, 'gfm_water', 'GFM slope class 1 (low slope)', filter_prop = 'slope_class', filter_value = 1)

In [ ]:
gfm_modloslo_allconf = confusion_matrix_reporter(gfm_swsl_allpoints, 'gfm_water', 'GFM slope class 2 (moderately low slope)', filter_prop = 'slope_class', filter_value = 2)

In [ ]:
gfm_modhislo_allconf = confusion_matrix_reporter(gfm_swsl_allpoints, 'gfm_water', 'GFM slope class 3 (moderately high slope)', filter_prop = 'slope_class', filter_value = 3)

### DSWx-HLS Slope -- All Confidences

In [ ]:
# DSWx-HLS Sampled with Slope Information
dswxhls_swsl_allpoints = sl_classes.sampleRegions(dswxhls_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale)

In [ ]:
dswxhls_loslo_allconf = confusion_matrix_reporter(dswxhls_swsl_allpoints, 'dswxhls_water', 'DSWx-HLS slope class 1 (low slope)', filter_prop = 'slope_class', filter_value = 1)

In [ ]:
dswxhls_modloslo_allconf = confusion_matrix_reporter(dswxhls_swsl_allpoints, 'dswxhls_water', 'DSWx-HLS slope class 2 (moderately low slope)', filter_prop = 'slope_class', filter_value = 2)

In [ ]:
dswxhls_modhislo_allconf = confusion_matrix_reporter(dswxhls_swsl_allpoints, 'dswxhls_water', 'DSWx-HLS slope class 3 (moderately high slope)', filter_prop = 'slope_class', filter_value = 3)

### DSWx-S1 Slope -- All Confidences

In [ ]:
# DSWx-S1 Sampled with Slope Information, all confidence points
dswxs1_swsl_allconf = sl_classes.sampleRegions(dswxs1_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale)

In [ ]:
dswxs1_loslo_allconf = confusion_matrix_reporter(dswxs1_swsl_allconf, 'dswxs1_water', 'DSWx-S1 slope class 1 (low slope)',
                                                 filter_prop = 'slope_class', filter_value = 1)

In [ ]:
dswxs1_modloslo_allconf = confusion_matrix_reporter(dswxs1_swsl_allconf, 'dswxs1_water', 'DSWx-S1 slope class 2 (moderately low slope)',
                                                    filter_prop = 'slope_class', filter_value = 2)

In [ ]:
dswxs1_modhislo_allconf = confusion_matrix_reporter(dswxs1_swsl_allconf, 'dswxs1_water', 'DSWx-S1 slope class 3 (moderately high slope)',
                                                    filter_prop = 'slope_class', filter_value = 3)

### MCDWD Slope -- All Confidences

In [ ]:
# MCDWD Sampled with Slope Information
mcdwd_swsl_allconf = sl_classes.sampleRegions(mcdwd_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale)

In [ ]:
# MCDWD Low Slope All Confidences
mcdwd_loslo_allconf = confusion_matrix_reporter(mcdwd_swsl_allpoints, 'mcdwd_water', 'DSWx-HLS slope class 1 (low slope)', filter_prop = 'slope_class', filter_value = 1)

In [ ]:
mcdwd_modloslo_allconf = confusion_matrix_reporter(mcdwd_swsl_allconf, 'mcdwd_water', 'MCDWD slope class 2 (moderately low slope)', filter_prop = 'slope_class', filter_value = 2)

In [ ]:
mcdwd_modhislo_allconf = confusion_matrix_reporter(mcdwd_swsl_allpoints, 'mcdwd_water', 'MCDWD slope class 3 (moderately high slope)', filter_prop = 'slope_class', filter_value = 3)

### VFM Slope -- All Confidences

In [ ]:
# VFM Sampled with Slope Information
vfm_swsl_allpoints = sl_classes.sampleRegions(vfm_sampled_allpoints, geometries=True, projection = my_proj_string, scale = my_scale)

In [ ]:
vfm_loslo_allconf = confusion_matrix_reporter(vfm_swsl_allpoints, 'vfm_water', 'VFM slope class 1 (low slope)', filter_prop = 'slope_class', filter_value = 1)

In [ ]:
vfm_modloslo_allconf = confusion_matrix_reporter(vfm_swsl_allpoints, 'vfm_water', 'VFM slope class 2 (moderately low slope)', filter_prop = 'slope_class', filter_value = 2)

In [ ]:
vfm_modhislo_allconf = confusion_matrix_reporter(vfm_swsl_allpoints, 'vfm_water', 'VFM slope class 3 (moderately high slope)', filter_prop = 'slope_class', filter_value = 3)

## Step 5 Part 2: Slope analysis of points with High Confidence Points

### HYDRAFloods Slope -- High Confidence

In [ ]:
hf_swsl_highconf = sl_classes.sampleRegions(hf_sampled_highconf, geometries=True, projection = my_proj_string, scale = my_scale) # hydrafloods sampled with slope information

In [ ]:
# HYDRAFloods Low Slope High Confidence
hf_loslo_highconf = confusion_matrix_reporter(hf_swsl_highconf, 'hydrafloods_water', 'HYDRAFloods Low Slope High Confidence)',
                                               filter_prop = 'slope_class', filter_value = 1)

In [ ]:
hf_modlowslo_highconf = confusion_matrix_reporter(hf_swsl_highconf, 'hydrafloods_water', 'HYDRAFloods Moderately Low Slope ',
                                                  filter_prop = 'slope_class', filter_value = 2)

In [ ]:
hf_modhislo_highconf = confusion_matrix_reporter(hf_swsl_highconf, 'hydrafloods_water', 'HYDRAFloods Moderately High Slope ',
                                                  filter_prop = 'slope_class', filter_value = 3)

### HYDROSAR Slope -- High Confidence

In [ ]:
hs_swsl_highconf = sl_classes.sampleRegions(hs_sampled_highconf, geometries=True,
                                          projection = my_proj_string, scale = my_scale) # hydrafloods sampled with slope information

In [ ]:
hs_lowslo_highconf = confusion_matrix_reporter(hs_swsl_highconf, 'hydrosar_water', 'HYDRAFloods Low Slope ',
                                                  filter_prop = 'slope_class', filter_value = 1)

In [ ]:
hs_modlowslo_highconf = confusion_matrix_reporter(hs_swsl_highconf, 'hydrosar_water', 'HYDRAFloods Moderately Low Slope ',
                                                  filter_prop = 'slope_class', filter_value = 2)

In [ ]:
hs_modhighslo_highconf = confusion_matrix_reporter(hs_swsl_highconf, 'hydrosar_water', 'HYDRAFloods High Slope ',
                                                  filter_prop = 'slope_class', filter_value = 3)

### GFM Slope -- High Confidence

In [ ]:
gfm_swsl_highconf = sl_classes.sampleRegions(gfm_sampled_highconf, geometries=True,
                                          projection = my_proj_string, scale = my_scale)

In [ ]:
gfm_lowslo_highconf = confusion_matrix_reporter(gfm_swsl_highconf, 'gfm_water', 'GFM Low Slope ',
                                                  filter_prop = 'slope_class', filter_value = 1)

In [ ]:
gfm_modlowslo_highconf = confusion_matrix_reporter(gfm_swsl_highconf, 'gfm_water', 'GFM Moderately Low Slope ',
                                                  filter_prop = 'slope_class', filter_value = 2)

In [ ]:
gfm_modhighslo_highconf = confusion_matrix_reporter(gfm_swsl_highconf, 'gfm_water', 'GFM Moderately High Slope',
                                                  filter_prop = 'slope_class', filter_value = 3)

### DSWx-HLS Slope -- High Confidence

In [ ]:
# DSWxHLS Sampled with Slope -- High Confidence Points Only
dswxhls_swsl_highconf = sl_classes.sampleRegions(dswxhls_sampled_highconf, geometries=True,
                                          projection = my_proj_string, scale = my_scale)

In [ ]:
dswxhls_lowslo_highconf = confusion_matrix_reporter(dswxhls_swsl_highconf, 'dswxhls_water', 'DSWx-HLS Low Slope ',
                                                  filter_prop = 'slope_class', filter_value = 1)

In [ ]:
dswxhls_modlowslo_highconf = confusion_matrix_reporter(dswxhls_swsl_highconf, 'dswxhls_water',
                                                       'DSWx-HLS Moderately Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 2)

In [ ]:
dswxhls_modhighslo_highconf = confusion_matrix_reporter(dswxhls_swsl_highconf, 'dswxhls_water',
                                                       'DSWx-HLS Moderately High Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 3)

### DSWx-S1 Slope -- High Confidence

In [ ]:
# DSWxS1 Sampled with Slope -- High Confidence Points Only
dswxs1_swsl_highconf = sl_classes.sampleRegions(dswxs1_sampled_highconf, geometries=True,
                                          projection = my_proj_string, scale = my_scale)

In [ ]:
dswxs1_lowslo_highconf = confusion_matrix_reporter(dswxs1_swsl_highconf, 'dswxs1_water',
                                                       'DSWx-S1 Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 1)

In [ ]:
dswxs1_modlowslo_highconf = confusion_matrix_reporter(dswxs1_swsl_highconf, 'dswxs1_water',
                                                       'DSWx-S1 Moderately Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 2)

In [ ]:
dswxs1_modhighslo_highconf = confusion_matrix_reporter(dswxs1_swsl_highconf, 'dswxs1_water',
                                                       'DSWx-S1 Moderately High Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 3)

### MCDWD Slope -- High Confidence

In [ ]:
# MCDWD Sampled with Slope -- High Confidence Points Only
mcdwd_swsl_highconf = sl_classes.sampleRegions(mcdwd_sampled_highconf, geometries=True,
                                          projection = my_proj_string, scale = my_scale)

In [ ]:
mcdwd_lowslo_highconf = confusion_matrix_reporter(mcdwd_swsl_highconf, 'mcdwd_water',
                                                       'MCDWD Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 1)

In [ ]:
mcdwd_modlowslo_highconf = confusion_matrix_reporter(mcdwd_swsl_highconf, 'mcdwd_water',
                                                       'MCDWD Moderately Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 2)

In [ ]:
mcdwd_modhighslo_highconf = confusion_matrix_reporter(mcdwd_swsl_highconf, 'mcdwd_water',
                                                       'MCDWD Moderately High Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 3)

### VFM Slope -- High Confidence

In [ ]:
# VFM Sampled with Slope -- High Confidence Points Only
vfm_swsl_highconf = sl_classes.sampleRegions(vfm_sampled_highconf, geometries=True,
                                          projection = my_proj_string, scale = my_scale)

In [ ]:
vfm_lowslo_highconf = confusion_matrix_reporter(vfm_swsl_highconf, 'vfm_water',
                                                       'VFM Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 1)

In [ ]:
vfm_modlowslo_highconf = confusion_matrix_reporter(vfm_swsl_highconf, 'vfm_water',
                                                       'VFM Moderately Low Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 2)

In [ ]:
vfm_modhighslo_highconf = confusion_matrix_reporter(vfm_swsl_highconf, 'vfm_water',
                                                       'VFM Moderately High Slope High Confidence',
                                                       filter_prop = 'slope_class', filter_value = 3)

# Step 6: Land Cover

Now, let's use the .sampleRegions() region to add the landcover information to the map.

In [ ]:
# Date at which the flood event starts
#doi = "2023-06-16"

# Get the date from a week prior
#week = ee.Date(doi).advance(-1, 'week')

# Get the date from six months prior
#month6 = ee.Date(doi).advance(-6, 'month')

szn = ee.Date(doi).advance(-3, 'month')

dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

#dw_filt = dw.filterBounds(aoi).filterDate(week, doi)

# Filter the Dynamic World Image Collection to the area of interest and to the time period from 6 months before the flood event starts
#dw_filt_6 = dw.filterBounds(aoi).filterDate(month6, doi)

# Filter the Dynamic World Image to the beginning of the prior season
dw_filt_seasonal = dw.filterBounds(aoi).filterDate(szn, doi).select(['label'])

# Apply a temporal median reducer at each pixel to dw_filt_6
#dw_redux_6 = dw_filt_6.mode()

dw_redux_sznl = dw_filt_seasonal.mode()

dw_discrete_sznl = dw_redux_sznl.clip(aoi)

In [ ]:
dw_discrete_sznl.bandNames()

In [ ]:
dw_renamed = dw_discrete_sznl.rename(["land_cover"])

In [ ]:
dw_renamed.projection().nominalScale().getInfo()

my_proj_string, my_scale

## Step 6 Part 1: Land Cover with All Confidences

In [ ]:
# HYDRAFloods sampled with Slope and Land Cover
hf_swlc_allpoints = dw_renamed.sampleRegions(hf_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# Hydrosar sampled with Land Cover
hs_swlc_allpoints = dw_renamed.sampleRegions(hs_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# GFM sampled with land cover
gfm_swlc_allpoints = dw_renamed.sampleRegions(gfm_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# DSWx-HLS Sampled with Land Cover
dswxhls_swlc_allpoints = dw_renamed.sampleRegions(dswxhls_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# DSWx-S1 sampled with land cover
dswxs1_swlc_allpoints = dw_renamed.sampleRegions(dswxs1_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# VFM sampled with land cover
vfm_swlc_allpoints = dw_renamed.sampleRegions(vfm_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# MCDWD sampled with land cover
mcdwd_swlc_allpoints = dw_renamed.sampleRegions(mcdwd_sampled_allpoints, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

In [ ]:
print(hs_swlc_allpoints.first().getInfo())

In [ ]:
#hf_swlc_all_poi.first().getInfo()

### HYDRAFloods Analysis by Land Cover, All Confidences

In [ ]:
#hf_cropland = accuracy_reporter(hf_swlc_allpoints, 'reference_water', 'hydrafloods_water', 'hydrafloods cropland all points', filter_bool=True, filter_prop='land_cover', filter_value=4)
hf_cropland_allconf = confusion_matrix_reporter(hf_swlc_allpoints, 'hydrafloods_water', 'HYDRAFloods cropland', filter_prop = 'land_cover', filter_value = 4)

In [ ]:
#hf_built = accuracy_reporter(hf_swlc_allpoints, 'reference_water', 'hydrafloods_water', 'hydrafloods built all points', filter_bool=True, filter_prop='land_cover', filter_value=6)
hf_built_allconf = confusion_matrix_reporter(hf_swlc_allpoints, 'hydrafloods_water', 'HYDRAFloods Built', filter_prop = 'land_cover', filter_value = 6)

In [ ]:
#hf_bare = accuracy_reporter(hf_swlc_allpoints, 'reference_water', 'hydrafloods_water', 'hydrafloods bare all points', filter_bool=True, filter_prop='land_cover', filter_value=7)
hf_bare_allconf = confusion_matrix_reporter(hf_swlc_allpoints, 'hydrafloods_water', 'HYDRAFloods bare', filter_prop = 'land_cover', filter_value = 7)

### HYDROSAR Analysis by Land Cover, All Confidences

In [ ]:
#hs_cropland_allpoints = accuracy_reporter(hs_swlc_all_points, 'reference_water', 'hydrosar_water', 'hydrosar cropland all points',
#                                filter_bool=True, filter_prop='land_cover', filter_value = 4)

hs_cropland_allconf = confusion_matrix_reporter(hs_swlc_allpoints, 'hydrosar_water',
                                                  'HYDROSAR Cropland',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
#hs_built_allpoints = accuracy_reporter(hs_swlc_all_points, 'reference_water', 'hydrosar_water', 'hydrosar built all points',
#                             filter_bool=True, filter_prop='land_cover', filter_value = 6)

hs_built_allconf = confusion_matrix_reporter(hs_swlc_allpoints, 'hydrosar_water', 'HYDROSAR Built All Points',
                                               filter_prop = 'land_cover', filter_value = 6)

In [ ]:
#hs_bare_allpoints = accuracy_reporter(hs_swlc_all_points, 'reference_water', 'hydrosar_water', 'hydrosar bare all points',
#                            filter_bool=True, filter_prop='land_cover', filter_value=7)
hs_bare_allconf = confusion_matrix_reporter(hs_swlc_allpoints, 'hydrosar_water', 'HYDROSAR Bare All Confidences',
                                               filter_prop = 'land_cover', filter_value = 7)

### GFM Analysis by Land Cover, All Confidences

In [ ]:
#gfm_cropland = accuracy_reporter(gfm_swlc_all_points, 'reference_water', 'gfm_water', 'GFM Cropland All points',
#                                 filter_bool=True, filter_prop='land_cover', filter_value = 4)

gfm_cropland_allconf = confusion_matrix_reporter(gfm_swlc_allpoints, 'gfm_water',
                                                  'GFM Cropland All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
#gfm_built_allpoints = accuracy_reporter(gfm_swlc_all_points, 'reference_water', 'gfm_water', 'GFM Built',
#                              filter_bool=True, filter_prop='land_cover', filter_value = 6)

gfm_built_allconf = confusion_matrix_reporter(gfm_swlc_allpoints, 'gfm_water',
                                                  'GFM Built All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
#gfm_bare_allpoints = accuracy_reporter(gfm_swlc_all_points, 'reference_water', 'gfm_water', 'GFM Bare',
#                              filter_bool=True, filter_prop='land_cover', filter_value = 7)

gfm_bare_allconf = confusion_matrix_reporter(gfm_swlc_allpoints, 'gfm_water',
                                                  'GFM Bare All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 7)

### DSWx-HLS Analysis by Land Cover, All Confidences

In [ ]:
#dswxhls_cropland = accuracy_reporter(dswxhls_swlc_all_points, 'reference_water', 'dswxhls_water', 'DSWx-HLS Cropland All confidences',
#                                     filter_bool=True, filter_prop='land_cover', filter_value = 4)

dswxhls_cropland_allconf = confusion_matrix_reporter(dswxhls_swlc_allpoints, 'dswxhls_water',
                                                  'DSWx-HLS Cropland All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
#dswxhls_built = accuracy_reporter(dswxhls_swlc_all_points, 'reference_water', 'dswxhls_water', 'DSWx-HLS Built All confidence',
#                                  filter_bool=True, filter_prop='land_cover', filter_value = 6)

dswxhls_built_allconf = confusion_matrix_reporter(dswxhls_swlc_allpoints, 'dswxhls_water',
                                                  'DSWx-HLS Built All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
#dswxhls_bare_allpoints = accuracy_reporter(dswxhls_swlc_all_points, 'reference_water', 'dswxhls_water', 'DSWx-HLS Bare All confidence',
#                              filter_bool=True, filter_prop='land_cover', filter_value = 7)

In [ ]:
dswxhls_bare_allconf = confusion_matrix_reporter(dswxhls_swlc_allpoints, 'dswxhls_water',
                                                  'DSWx-HLS Bare All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 7)

### DSWx-S1 Analysis by Land Cover, All Confidences

In [ ]:
#dswxs1_cropland = accuracy_reporter(dswxs1_swlc_all_points, 'reference_water', 'dswxs1_water', 'DSWx-S1 Cropland All confidences',
#                                     filter_bool=True, filter_prop='land_cover', filter_value = 4)

dswxs1_cropland_allconf = confusion_matrix_reporter(dswxs1_swlc_allpoints, 'dswxs1_water',
                                                  'DSWx-S1 Cropland All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
#dswxs1_built = accuracy_reporter(dswxs1_swlc_all_points, 'reference_water', 'dswxhls_water', 'DSWx-S1 Built All confidence',
#                                  filter_bool=True, filter_prop='land_cover', filter_value = 6)

In [ ]:
dswxs1_built_allconf = confusion_matrix_reporter(dswxs1_swlc_allpoints, 'dswxs1_water',
                                                  'DSWx-S1 Built All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
#dswxs1_bare_allpoints = accuracy_reporter(dswxs1_swlc_all_points, 'reference_water', 'dswxs1_water', 'DSWx-S1 Bare All confidence',
#                              filter_bool=True, filter_prop='land_cover', filter_value = 7)

In [ ]:
dswxs1_bare_allconf = confusion_matrix_reporter(dswxs1_swlc_allpoints, 'dswxs1_water',
                                                  'DSWx-S1 Bare All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 7)

### MCDWD Analysis by Land Cover, All Confidences

In [ ]:
#mcdwd_cropland_all_points = accuracy_reporter(mcdwd_swlc_all_points, 'reference_water', 'mcdwd_water', 'MCDWD Cropland All points',
#                                   filter_bool=True, filter_prop='land_cover', filter_value = 4)

#mcdwd_built_all_points = accuracy_reporter(mcdwd_swlc_all_points, 'reference_water', 'mcdwd_water', 'MCDWD Built All points',
#                                           filter_bool=True, filter_prop='land_cover', filter_value = 6)

#mcdwd_bare_allpoints = accuracy_reporter(mcdwd_swlc_all_points, 'reference_water', 'mcdwd_water', 'MCDWD Bare',
#                              filter_bool=True, filter_prop='land_cover', filter_value = 7)

In [ ]:
mcdwd_cropland_allconf = confusion_matrix_reporter(mcdwd_swlc_allpoints, 'mcdwd_water',
                                                  'MCDWD Cropland All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
mcdwd_built_allconf = confusion_matrix_reporter(mcdwd_swlc_allpoints, 'mcdwd_water',
                                                  'MCDWD Built All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
mcdwd_bare_allconf = confusion_matrix_reporter(mcdwd_swlc_allpoints, 'mcdwd_water',
                                                  'MCDWD Bare Ground All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 7)

### VFM Analysis by Land Cover, All Confidences

In [ ]:
'''
vfm_cropland_allpoints = accuracy_reporter(vfm_swlc_all_points, 'reference_water', 'vfm_water', 'VFM Cropland All points',
                                           filter_bool=True, filter_prop='land_cover', filter_value = 4)
vfm_built_allpoints = accuracy_reporter(vfm_swlc_all_points, 'reference_water', 'vfm_water', 'VFM Built all points',
                                        filter_bool=True, filter_prop='land_cover', filter_value = 6)
vfm_bare_allpoints = accuracy_reporter(vfm_swlc_all_points, 'reference_water', 'vfm_water', 'VFM Bare All confidence',
                              filter_bool=True, filter_prop='land_cover', filter_value = 7)
'''

In [ ]:
vfm_cropland_allconf = confusion_matrix_reporter(vfm_swlc_allpoints, 'vfm_water',
                                                  'VFM Cropland All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
vfm_built_allconf = confusion_matrix_reporter(vfm_swlc_allpoints, 'vfm_water',
                                                  'VFM Built-Up All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
vfm_bare_allconf = confusion_matrix_reporter(vfm_swlc_allpoints, 'vfm_water',
                                                  'VFM Bare Ground All Confidences',
                                                  filter_prop = 'land_cover', filter_value = 7)

## Step 6 Part 2: Land Cover with High Confidence Points

In [ ]:
# HYDRAFloods sampled with Slope, High Confidence
hf_swlc_highconf = dw_renamed.sampleRegions(hf_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# Hydrosar sampled with Land Cover, High Confidence
hs_swlc_highconf = dw_renamed.sampleRegions(hs_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# GFM sampled with land cover, High Confidence
gfm_swlc_highconf = dw_renamed.sampleRegions(gfm_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# DSWx-HLS Sampled with Land Cover, High Confidence
dswxhls_swlc_highconf = dw_renamed.sampleRegions(dswxhls_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# DSWx-S1 sampled with land cover, High Confidence
dswxs1_swlc_highconf = dw_renamed.sampleRegions(dswxs1_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# VFM sampled with land cover, High Confidence
vfm_swlc_highconf = dw_renamed.sampleRegions(vfm_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

# MCDWD sampled with land cover, High Confidence
mcdwd_swlc_highconf = dw_renamed.sampleRegions(mcdwd_sampled_highconf, projection = my_proj_string,
                                            scale=my_scale, geometries = True)

### HYDRAFloods Analysis by Land Cover, High Confidence Points Only

In [ ]:
hf_cropland_highconf = confusion_matrix_reporter(hf_swlc_highconf, 'hydrafloods_water',
                                                  'HYDRAFloods Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
hf_built_highconf = confusion_matrix_reporter(hf_swlc_highconf, 'hydrafloods_water',
                                                  'HYDRAFloods Built High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
hf_bare_highconf = confusion_matrix_reporter(hf_swlc_highconf, 'hydrafloods_water',
                                                  'HYDRAFloods Bare High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

### HYDROSAR Analysis by Land Cover, High Confidence Points Only

In [ ]:
hs_cropland_highconf = confusion_matrix_reporter(hs_swlc_highconf, 'hydrosar_water',
                                                  'HYDROSAR Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
hs_built_highconf = confusion_matrix_reporter(hs_swlc_highconf, 'hydrosar_water',
                                                  'HYDROSAR Built-Up High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
hs_bare_highconf = confusion_matrix_reporter(hs_swlc_highconf, 'hydrosar_water',
                                                  'HYDROSAR Bare Ground High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

### GFM Analysis by Land Cover, High Confidence Points Only

In [ ]:
gfm_cropland_highconf = confusion_matrix_reporter(gfm_swlc_highconf, 'gfm_water',
                                                  'GFM Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
gfm_built_highconf = confusion_matrix_reporter(gfm_swlc_highconf, 'gfm_water',
                                                  'GFM Built-Up High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
gfm_bare_highconf = confusion_matrix_reporter(gfm_swlc_highconf, 'gfm_water',
                                                  'GFM Bare Ground High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

### DSWx-HLS Analysis by Land Cover, High Confidence Points Only

In [ ]:
dswxhls_cropland_highconf = confusion_matrix_reporter(dswxhls_swlc_highconf, 'dswxhls_water',
                                                  'DSWx-HLS Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
dswxhls_built_highconf = confusion_matrix_reporter(dswxhls_swlc_highconf, 'dswxhls_water',
                                                  'DSWx-HLS Built High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
dswxhls_bare_highconf = confusion_matrix_reporter(dswxhls_swlc_highconf, 'dswxhls_water',
                                                  'DSWx-HLS Bare Ground High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

### DSWx-S1 Analysis by Land Cover, High Confidence Points Only

In [ ]:
dswxs1_cropland_highconf = confusion_matrix_reporter(dswxs1_swlc_highconf, 'dswxs1_water',
                                                  'DSWx-S1 Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
dswxs1_built_highconf = confusion_matrix_reporter(dswxs1_swlc_highconf, 'dswxs1_water',
                                                  'DSWx-S1 Built High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
dswxs1_bare_highconf = confusion_matrix_reporter(dswxs1_swlc_highconf, 'dswxs1_water',
                                                  'DSWx-S1 Bare Ground High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

### MCDWD Analysis by Land Cover, High Confidence Points Only

In [ ]:
mcdwd_cropland_highconf = confusion_matrix_reporter(mcdwd_swlc_highconf, 'mcdwd_water',
                                                  'MCDWD Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
mcdwd_built_highconf = confusion_matrix_reporter(mcdwd_swlc_highconf, 'mcdwd_water',
                                                  'MCDWD Built-Up High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
mcdwd_bare_highconf = confusion_matrix_reporter(mcdwd_swlc_highconf, 'mcdwd_water',
                                                  'MCDWD Bare Ground High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

### VFM Analysis by Land Cover, High Confidence Points Only

In [ ]:
vfm_cropland_highconf = confusion_matrix_reporter(vfm_swlc_highconf, 'vfm_water',
                                                  'VFM Cropland High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 4)

In [ ]:
vfm_built_highconf = confusion_matrix_reporter(vfm_swlc_highconf, 'vfm_water',
                                                  'VFM Built-Up High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 6)

In [ ]:
vfm_bare_highconf = confusion_matrix_reporter(vfm_swlc_highconf, 'vfm_water',
                                                  'VFM Bare Ground High Confidence',
                                                  filter_prop = 'land_cover', filter_value = 7)

# Step 7: Write everything to a CSV

In [ ]:
vfm_bare_highconf

In [ ]:
pwd

In [ ]:
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive/')

In [ ]:
os.chdir('drive/My Drive')#/Flood_Intercomparison/case_studies/confirmed_case_studies/chad_09_26')

In [ ]:
os.chdir('Flood_Intercomparison')

In [ ]:
os.chdir('Case_Studies/confirmed_case_studies/chad_09_26')

In [ ]:
import csv

In [ ]:
#filename = 'Statistics_chad_09_26_20250326.csv'
filename = 'Statistics_chad_09_26_20250326_updated.csv'

headers = ['Description', 'Sample_Size', 'Overall_Accuracy', 'Precision', 'Recall', 'Specificity', 'NPV', 'F1_score', 'Normalized_Matthews_Correlation_Coefficient']

with open(filename, 'w') as csvfile:
  csvwriter = csv.writer(csvfile)

  csvwriter.writerow(headers)

  # All Confidence Points
  csvwriter.writerow(hydrafloods_allconf)
  csvwriter.writerow(hydrosar_allconf)
  csvwriter.writerow(gfm_allconf)
  csvwriter.writerow(dswxhls_allconf)
  csvwriter.writerow(dswxs1_allconf)
  csvwriter.writerow(mcdwd_allconf)
  csvwriter.writerow(vfm_allconf)

  # High Confidence Points
  csvwriter.writerow(hydrafloods_highconf)
  csvwriter.writerow(hydrosar_highconf)
  csvwriter.writerow(gfm_highconf)
  csvwriter.writerow(dswxhls_highconf)
  csvwriter.writerow(dswxs1_highconf)
  csvwriter.writerow(mcdwd_highconf)
  csvwriter.writerow(vfm_highconf)

  # Slope -- All Confidence Points
  csvwriter.writerow(hf_lowslope_allconf)
  csvwriter.writerow(hs_lowslo_allconf)
  csvwriter.writerow(gfm_loslo_allconf)
  csvwriter.writerow(dswxhls_loslo_allconf)
  csvwriter.writerow(dswxs1_loslo_allconf)
  csvwriter.writerow(mcdwd_loslo_allconf)
  csvwriter.writerow(vfm_loslo_allconf)

  csvwriter.writerow(hf_modlowslope_allconf)
  csvwriter.writerow(hs_modlowslo_allconf)
  csvwriter.writerow(gfm_modloslo_allconf)
  csvwriter.writerow(dswxhls_modloslo_allconf)
  csvwriter.writerow(dswxs1_modloslo_allconf)
  csvwriter.writerow(mcdwd_modloslo_allconf)
  csvwriter.writerow(vfm_modloslo_allconf)


  csvwriter.writerow(hf_modhighslope_allconf)
  csvwriter.writerow(hs_modhighslo_allconf)
  csvwriter.writerow(gfm_modhislo_allconf)
  csvwriter.writerow(dswxhls_modhislo_allconf)
  csvwriter.writerow(dswxs1_modhislo_allconf)
  csvwriter.writerow(mcdwd_modhislo_allconf)
  csvwriter.writerow(vfm_modhislo_allconf)

  # Slope -- High Confidence Points
  csvwriter.writerow(hf_loslo_highconf)
  csvwriter.writerow(hs_lowslo_highconf)
  csvwriter.writerow(gfm_lowslo_highconf)
  csvwriter.writerow(dswxhls_lowslo_highconf)
  csvwriter.writerow(dswxs1_lowslo_highconf)
  csvwriter.writerow(mcdwd_lowslo_highconf)
  csvwriter.writerow(vfm_lowslo_highconf)

  csvwriter.writerow(hf_modlowslo_highconf)
  csvwriter.writerow(hs_modlowslo_highconf)
  csvwriter.writerow(gfm_modlowslo_highconf)
  csvwriter.writerow(dswxhls_modlowslo_highconf)
  csvwriter.writerow(dswxs1_modlowslo_highconf)
  csvwriter.writerow(mcdwd_modlowslo_highconf)
  csvwriter.writerow(vfm_modlowslo_highconf)

  csvwriter.writerow(hf_modhislo_highconf)
  csvwriter.writerow(hs_modhighslo_highconf)
  csvwriter.writerow(gfm_modhighslo_highconf)
  csvwriter.writerow(dswxhls_modhighslo_highconf)
  csvwriter.writerow(dswxs1_modhighslo_highconf)
  csvwriter.writerow(mcdwd_modhighslo_highconf)
  csvwriter.writerow(vfm_modhighslo_highconf)

  # Land Cover -- All Confidence Points
  csvwriter.writerow(hf_cropland_allconf)
  csvwriter.writerow(hs_cropland_allconf)
  csvwriter.writerow(gfm_cropland_allconf)
  csvwriter.writerow(dswxhls_cropland_allconf)
  csvwriter.writerow(dswxs1_cropland_allconf)
  csvwriter.writerow(mcdwd_cropland_allconf)
  csvwriter.writerow(vfm_cropland_allconf)

  csvwriter.writerow(hf_built_allconf)
  csvwriter.writerow(hs_built_allconf)
  csvwriter.writerow(gfm_built_allconf)
  csvwriter.writerow(dswxhls_built_allconf)
  csvwriter.writerow(dswxs1_built_allconf)
  csvwriter.writerow(mcdwd_built_allconf)
  csvwriter.writerow(vfm_built_allconf)

  csvwriter.writerow(hf_bare_allconf)
  csvwriter.writerow(hs_bare_allconf)
  csvwriter.writerow(gfm_bare_allconf)
  csvwriter.writerow(dswxhls_bare_allconf)
  csvwriter.writerow(dswxs1_bare_allconf)
  csvwriter.writerow(mcdwd_bare_allconf)
  csvwriter.writerow(vfm_bare_allconf)

  # Land cover -- High Confidence Points
  csvwriter.writerow(hf_cropland_highconf)
  csvwriter.writerow(hs_cropland_highconf)
  csvwriter.writerow(gfm_cropland_highconf)
  csvwriter.writerow(dswxhls_cropland_highconf)
  csvwriter.writerow(dswxs1_cropland_highconf)
  csvwriter.writerow(mcdwd_cropland_highconf)
  csvwriter.writerow(vfm_cropland_highconf)

  csvwriter.writerow(hf_built_highconf)
  csvwriter.writerow(hs_built_highconf)
  csvwriter.writerow(gfm_built_highconf)
  csvwriter.writerow(dswxhls_built_highconf)
  csvwriter.writerow(dswxs1_built_highconf)
  csvwriter.writerow(mcdwd_built_highconf)
  csvwriter.writerow(vfm_built_highconf)

  csvwriter.writerow(hf_bare_highconf)
  csvwriter.writerow(hs_bare_highconf)
  csvwriter.writerow(gfm_bare_highconf)
  csvwriter.writerow(dswxhls_bare_highconf)
  csvwriter.writerow(dswxs1_bare_highconf)
  csvwriter.writerow(mcdwd_bare_highconf)
  csvwriter.writerow(vfm_bare_highconf)

# References

* Landwehr, Tim, Antara Dasgupta, and Björn Waske. "Towards robust validation strategies for EO flood maps." Remote Sensing of Environment 315 (2024): 114439.